Importing NFL Reciever Data

Data Sources
    - (NOT SURE YOU CAN PULL FROM HERE) https://nextgenstats.nfl.com/stats/receiving#percent-intended-yards
    - https://fantasydata.com/nfl/fantasy-football-leaders?season=2023&seasontype=1&scope=1&subscope=1&scoringsystem=2&aggregatescope=1&range=3
    -https://premium.pff.com/nfl/positions/2023/REGPO/receiving?position=WR,TE,RB,ORCV
    - https://nflsavant.com/targets.php?ddlYear=2022&ddlPosition=&week=&rz=redzone&ddlTeam=
    - https://www.pro-football-reference.com/years/2023/fantasy.htm

Final Data Sources
    - pff_receiving: https://premium.pff.com/nfl/positions/2023/REG/receiving-scheme?minimum=1
    - pff_man_zone: https://premium.pff.com/nfl/positions/2023/REG/receiving
    - pfr_fantasy: https://www.pro-football-reference.com/years/2023/fantasy.htm
    - pfr_rz: https://www.pro-football-reference.com/years/2023/redzone-receiving.htm
    

Problems
1. Rookies
2. Injuries
    - Could do most previous non injured season
    - Need to do everything per game
3. New teams
4. New coaches/schemes

** Overall Ideas **
- Maybe it would be interesting to value players by projected points above average (of position) and then have the AI do mock drafts to look at the strategy?


**Variable Ideas**

- Last year personal stats:
    - Fantasy Points per game (pfr)
        - Need to make sure scoring matches
    - Target Share
        - Need to somehow make this per game
        - Maybe just do Targets per game (PFF [pff_receiving])
        - Redzone Targets per game (pfr [pfr_rz])
    - Yards/Game (PFF [pff_receiving])
    - Catch Rate (PFF [pff_receiving])
    - Yards after catch (average) (PFF [pff_receiving])
    - TD's (Expected TD's) (per game) (PFF [pff_receiving])
    - Efficiency against Man (PFF [pff_man_zone])
    - Efficiency against Zone (PFF [pff_man_zone])
    - Average Seperation/Cushion? (Next Gen Stats Don't think you can download)
    - Average Depth of Target (PFF [pff_receiving])
    - % Slot/Outside (PFF [pff_receiving])
    - Number/% of Snaps (PFF [pff_receiving])
    - Position in Depth Chart
    - RECV (PFF [pff_receiving])
    - Yards / Routes Run (PFF [pff_receiving])
    - Routes Run per game


- Last year (new) team stats:
    - Offense rating (PFF Don't think you can download)
    - Pass Volume
    - QB Ranking
    - O-Line Pass Block Ranking (PFF Don't think you can download)
    - Time of Possession

- This year personal stats:
    - Age/Experience
    - Position in Depth Chart

- Overall Status
    - % increase/decrease in PPG from last year (PFF [pff_receiving])

- This year team stats:
    - Vacant Target Share



Reading in pff_receiving data

In [1]:
import pandas as pd

# Reading in pff_receiving data

pr14_15 = pd.read_csv('pff_receiving_14-15.csv')
pr15_16 = pd.read_csv('pff_receiving_15-16.csv')
pr16_17 = pd.read_csv('pff_receiving_16-17.csv')
pr17_18 = pd.read_csv('pff_receiving_17-18.csv')
pr18_19 = pd.read_csv('pff_receiving_18-19.csv')
pr19_20 = pd.read_csv('pff_receiving_19-20.csv')
pr20_21 = pd.read_csv('pff_receiving_20-21.csv')
pr21_22 = pd.read_csv('pff_receiving_21-22.csv')
pr22_23 = pd.read_csv('pff_receiving_22-23.csv')
pr23_24 = pd.read_csv('pff_receiving_23-24.csv')

# Creating a season column to differentiate between seasons

pr14_15['Season'] = '2014-15'
pr15_16['Season'] = '2015-16'
pr16_17['Season'] = '2016-17'
pr17_18['Season'] = '2017-18'
pr18_19['Season'] = '2018-19'
pr19_20['Season'] = '2019-20'
pr20_21['Season'] = '2020-21'
pr21_22['Season'] = '2021-22'
pr22_23['Season'] = '2022-23'
pr23_24['Season'] = '2023-24'

# Combining all of the different season's data into one pff_receiving dataset

pff_receiving = pd.concat([pr14_15, pr15_16, pr16_17, pr17_18, pr18_19, pr19_20, pr20_21, pr21_22, pr22_23, pr23_24])

# Creating PPR PPG column to calculate fantasy points for a season. NEED TO CORRECT WITH ALL INFO NOT JUST REC

pff_receiving['PPR PPG'] = (pff_receiving['touchdowns'] * 6 + (pff_receiving['yards'] / 10) + pff_receiving['receptions']) / pff_receiving['player_game_count']

# Filter on only Wide Recievers

pff_receiving = pff_receiving[pff_receiving['position'] == 'WR']

pff_receiving

,player,player_id,position,team_name,player_game_count,avg_depth_of_target,avoided_tackles,caught_percent,contested_catch_rate,contested_receptions,...,touchdowns,wide_rate,wide_snaps,yards,yards_after_catch,yards_after_catch_per_reception,yards_per_reception,yprr,Season,PPR PPG
0,Antonio Brown,5718,WR,PIT,16,10.4,17,72.5,NaN,0,...,13,82.1,552,1698,615,4.8,13.2,2.67,2014-15,23.550000
1,Demaryius Thomas,5547,WR,DEN,16,10.5,11,62.4,NaN,0,...,11,80.7,505,1619,668,6.0,14.6,2.78,2014-15,21.181250
2,Julio Jones,6158,WR,ATL,15,12.4,16,66.7,NaN,0,...,6,86.7,534,1593,553,5.3,15.3,2.72,2014-15,19.953333
3,Jordy Nelson,4350,WR,GB,16,12.0,14,67.1,NaN,0,...,13,79.2,480,1519,519,5.3,15.5,2.66,2014-15,20.493750
4,Kelvin Benjamin,8663,WR,CAR,16,14.9,9,51.4,NaN,0,...,9,87.4,500,1008,177,2.4,13.8,1.86,2014-15,14.237500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,Trent Taylor,11932,WR,CHI,13,4.0,0,0.0,0.0,0,...,0,36.9,24,0,0,NaN,NaN,0.00,2023-24,0.000000
466,Kwamie Lassiter II,23226,WR,CIN,1,-2.0,0,100.0,NaN,0,...,0,100.0,2,2,4,4.0,2.0,1.00,2023-24,1.200000
475,Shedrick Jackson,84136,WR,CIN,1,31.0,0,0.0,0.0,0,...,0,66.7,2,0,0,NaN,NaN,0.00,2023-24,0.000000
476,Collin Johnson,47808,WR,CHI,3,7.0,0,100.0,NaN,0,...,0,60.0,9,11,4,4.0,11.0,0.79,2023-24,0.700000


Reading in the pff_man_zone data

In [2]:
pmz14_15 = pd.read_csv('pff_man_zone_14-15.csv')
pmz15_16 = pd.read_csv('pff_man_zone_15-16.csv')
pmz16_17 = pd.read_csv('pff_man_zone_16-17.csv')
pmz17_18 = pd.read_csv('pff_man_zone_17-18.csv')
pmz18_19 = pd.read_csv('pff_man_zone_18-19.csv')
pmz19_20 = pd.read_csv('pff_man_zone_19-20.csv')
pmz20_21 = pd.read_csv('pff_man_zone_20-21.csv')
pmz21_22 = pd.read_csv('pff_man_zone_21-22.csv')
pmz22_23 = pd.read_csv('pff_man_zone_22-23.csv')
pmz23_24 = pd.read_csv('pff_man_zone_23-24.csv')

pmz14_15['Season'] = '2014-15'
pmz15_16['Season'] = '2015-16'
pmz16_17['Season'] = '2016-17'
pmz17_18['Season'] = '2017-18'
pmz18_19['Season'] = '2018-19'
pmz19_20['Season'] = '2019-20'
pmz20_21['Season'] = '2020-21'
pmz21_22['Season'] = '2021-22'
pmz22_23['Season'] = '2022-23'
pmz23_24['Season'] = '2023-24'

pff_man_zone = pd.concat([pmz14_15, pmz15_16, pmz16_17, pmz17_18, pmz18_19, pmz19_20, pmz20_21, pmz21_22, pmz22_23, pmz23_24])

pff_man_zone




,player,player_id,position,team_name,player_game_count,base_targets,declined_penalties,franchise_id,man_avg_depth_of_target,man_avoided_tackles,...,zone_targeted_qb_rating,zone_targets,zone_targets_percent,zone_touchdowns,zone_yards,zone_yards_after_catch,zone_yards_after_catch_per_reception,zone_yards_per_reception,zone_yprr,Season
0,Antonio Brown,5718,WR,PIT,16,178,0,25,10.4,10,...,98.0,101,25.8,1,964,335,4.5,13.0,2.46,2014-15
1,Demaryius Thomas,5547,WR,DEN,16,178,0,10,10.0,5,...,100.2,102,28.0,4,1023,379,5.6,15.0,2.81,2014-15
2,Julio Jones,6158,WR,ATL,15,156,0,2,15.0,4,...,101.0,109,25.6,3,1107,386,5.0,14.4,2.60,2014-15
3,Jordy Nelson,4350,WR,GB,16,146,0,12,11.0,6,...,124.1,86,24.5,6,938,274,4.6,15.6,2.67,2014-15
4,Kelvin Benjamin,8663,WR,CAR,16,142,0,5,12.5,2,...,87.5,88,26.0,5,703,109,2.4,15.6,2.07,2014-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,Rashaad Penny,27414,HB,PHI,2,1,0,24,NaN,0,...,87.5,1,25.0,0,5,6,6.0,5.0,1.25,2023-24
478,Cody Thompson,48152,WR,SEA,6,1,0,29,NaN,0,...,108.3,1,33.3,0,10,3,3.0,10.0,3.33,2023-24
479,Khalil Dorsey,27819,CB,DET,1,1,0,11,NaN,0,...,0.0,0,NaN,0,0,0,NaN,NaN,NaN,2023-24
480,Mark Glowinski,9567,G,NYG,12,1,0,21,NaN,0,...,0.0,0,NaN,0,0,0,NaN,NaN,NaN,2023-24


Reading in pfr_rz data

In [3]:
pfr_rz14_15 = pd.read_csv('pfr_rz_14-15.csv')
pfr_rz15_16 = pd.read_csv('pfr_rz_15-16.csv')
pfr_rz16_17 = pd.read_csv('pfr_rz_16-17.csv')
pfr_rz17_18 = pd.read_csv('pfr_rz_17-18.csv')
pfr_rz18_19 = pd.read_csv('pfr_rz_18-19.csv')
pfr_rz19_20 = pd.read_csv('pfr_rz_19-20.csv')
pfr_rz20_21 = pd.read_csv('pfr_rz_20-21.csv')
pfr_rz21_22 = pd.read_csv('pfr_rz_21-22.csv')
pfr_rz22_23 = pd.read_csv('pfr_rz_22-23.csv')
pfr_rz23_24 = pd.read_csv('pfr_rz_23-24.csv')

pfr_rz14_15['Season'] = '2014-15'
pfr_rz15_16['Season'] = '2015-16'
pfr_rz16_17['Season'] = '2016-17'
pfr_rz17_18['Season'] = '2017-18'
pfr_rz18_19['Season'] = '2018-19'
pfr_rz19_20['Season'] = '2019-20'
pfr_rz20_21['Season'] = '2020-21'
pfr_rz21_22['Season'] = '2021-22'
pfr_rz22_23['Season'] = '2022-23'
pfr_rz23_24['Season'] = '2023-24'

pfr_rz = pd.concat([pfr_rz14_15, pfr_rz15_16, pfr_rz16_17, pfr_rz17_18, pfr_rz18_19, pfr_rz19_20, pfr_rz20_21, pfr_rz21_22, pfr_rz22_23, pfr_rz23_24])

pfr_rz

,Unnamed: 0,Unnamed: 1,Inside 20,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Inside 10,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Season
0,Player,Tm,Tgt,Rec,Ctch%,Yds,TD,%Tgt,Tgt,Rec,Ctch%,Yds,TD,%Tgt,Link,2014-15
1,Demaryius Thomas,DEN,39,18,46.15%,137,6,36.80%,18,6,33.33%,33,4,33.30%,All Demaryius Thomas red zone receiving plays,2014-15
2,Antonio Brown,PIT,33,18,54.55%,133,8,36.70%,18,9,50.00%,45,6,36.00%,All Antonio Brown red zone receiving plays,2014-15
3,Andre Johnson,HOU,26,12,46.15%,54,3,38.80%,11,5,45.45%,25,3,37.90%,All Andre Johnson red zone receiving plays,2014-15
4,Odell Beckham,NYG,25,16,64.00%,122,8,25.50%,10,6,60.00%,42,6,19.20%,All Odell Beckham red zone receiving plays,2014-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,Jamaal Williams,NOR,1,1,100.00%,3,0,1.50%,0,0,NaN,0,0,0.00%,All Jamaal Williams red zone receiving plays,2023-24
346,Trayveon Williams,CIN,1,1,100.00%,3,0,1.40%,0,0,NaN,0,0,0.00%,All Trayveon Williams red zone receiving plays,2023-24
347,Jeff Wilson,MIA,1,0,0.00%,0,0,1.50%,1,0,0.00%,0,0,2.30%,All Jeff Wilson red zone receiving plays,2023-24
348,Charlie Woerner,SFO,1,1,100.00%,3,0,1.40%,0,0,NaN,0,0,0.00%,All Charlie Woerner red zone receiving plays,2023-24


Reading in pfr_fp data

In [10]:
pfr_fp14_15 = pd.read_csv('pfr_fp_14-15.csv')
pfr_fp15_16 = pd.read_csv('pfr_fp_15-16.csv')
pfr_fp16_17 = pd.read_csv('pfr_fp_16-17.csv')
pfr_fp17_18 = pd.read_csv('pfr_fp_17-18.csv')
pfr_fp18_19 = pd.read_csv('pfr_fp_18-19.csv')
pfr_fp19_20 = pd.read_csv('pfr_fp_19-20.csv')
pfr_fp20_21 = pd.read_csv('pfr_fp_20-21.csv')
pfr_fp21_22 = pd.read_csv('pfr_fp_21-22.csv')
pfr_fp22_23 = pd.read_csv('pfr_fp_22-23.csv')
pfr_fp23_24 = pd.read_csv('pfr_fp_23-24.csv')

pfr_fp14_15['Season'] = '2014-15'
pfr_fp15_16['Season'] = '2015-16'
pfr_fp16_17['Season'] = '2016-17'
pfr_fp17_18['Season'] = '2017-18'
pfr_fp18_19['Season'] = '2018-19'
pfr_fp19_20['Season'] = '2019-20'
pfr_fp20_21['Season'] = '2020-21'
pfr_fp21_22['Season'] = '2021-22'
pfr_fp22_23['Season'] = '2022-23'
pfr_fp23_24['Season'] = '2023-24'

pfr_fp = pd.concat([pfr_fp14_15, pfr_fp15_16, pfr_fp16_17, pfr_fp17_18, pfr_fp18_19, pfr_fp19_20, pfr_fp20_21, pfr_fp21_22, pfr_fp22_23, pfr_fp23_24])

pfr_fp['Player'] = pfr_fp['Player'].str.replace(r'[*+]', '')

pfr_fp

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank,-9999,Season
0,1,DeMarco Murray*+,DAL,RB,26,16,16,0,0,0,...,NaN,294.0,351.1,362.1,322.6,173.0,1,1.0,MurrDe00,2014-15
1,2,Le'Veon Bell*+,PIT,RB,22,16,16,0,0,0,...,NaN,288.0,370.5,376.5,329.0,167.0,2,2.0,BellLe00,2014-15
2,3,Marshawn Lynch*,SEA,RB,28,16,14,0,0,0,...,NaN,265.0,302.3,310.3,283.8,144.0,3,3.0,LyncMa00,2014-15
3,4,Antonio Brown*+,PIT,WR,26,16,16,2,2,20,...,NaN,258.0,386.9,391.9,322.4,136.0,1,4.0,BrowAn04,2014-15
4,5,Matt Forte,CHI,RB,29,16,16,0,0,0,...,NaN,245.0,346.6,354.6,295.6,124.0,4,5.0,FortMa00,2014-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627,628,Kyle Allen,BUF,QB,27,7,0,0,0,0,...,NaN,-1.0,-1.3,-1.3,-1.3,NaN,83,NaN,AlleKy00,2023-24
628,629,Deon Jackson,3TM,RB,24,4,1,0,0,0,...,NaN,-1.0,4.0,6.0,1.5,NaN,166,NaN,JackDe02,2023-24
629,630,David Wells,TAM,TE,28,5,0,0,0,0,...,NaN,-1.0,1.0,1.0,NaN,NaN,140,NaN,WellDa00,2023-24
630,631,James Proche,CLE,WR,27,10,1,0,0,0,...,NaN,-2.0,-2.0,-1.0,-2.0,NaN,242,NaN,ProcJa00,2023-24


In [5]:
df = pff_receiving[pff_receiving['Season'] == '2023-24']
df[['player', 'PPR PPG']].sort_values(['PPR PPG'], ascending=False).head(20)

,player,PPR PPG
2,Tyreek Hill,23.556250
0,CeeDee Lamb,22.464706
9,Keenan Allen,21.100000
5,Amon-Ra St. Brown,20.656250
38,Justin Jefferson,20.540000
7,A.J. Brown,17.270588
28,Nico Collins,17.180000
6,Puka Nacua,17.035294
217,Mike Williams,16.633333
18,Mike Evans,16.617647
